<a href="https://colab.research.google.com/github/JasperLS/EWF/blob/main/UE_EWF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Übung 5**

In [ ]:
pip install researchpy

In [ ]:
import pandas as pd
import researchpy as rp

In [ ]:
# Einlesen der Datei
df = pd.read_csv('https://raw.githubusercontent.com/JasperLS/EWF/main/datasets/UE_Bier.csv',index_col=0)

In [ ]:
df.sample()

In [ ]:
# t-Test von Geschlecht auf Bierausgaben ausführen
summary_a, results_a = rp.ttest(group1 = df.loc[df['männlich'] == 1, 'bierausgaben'], group1_name = 'männlich', 
         group2 = df.loc[df['männlich'] == 0, 'bierausgaben'], group2_name = 'weiblich')

In [ ]:
summary_a

In [ ]:
results_a

In [ ]:
# t-Test von Bildungsgrad auf Einkommen ausführen
summary_b, results_b = rp.ttest(group1 = df.loc[df.bildungsgrad == 1,'einkommen'], group1_name = 'bildungsgrad = 1', 
         group2 = df.loc[df.bildungsgrad == 4,'einkommen'], group2_name = 'bildungsgrad = 4')

In [ ]:
summary_b

In [ ]:
results_b

## **Übung 6**


In [ ]:
pip install bioinfokit

In [ ]:
import pandas as pd
import scipy
from bioinfokit.analys import stat
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/JasperLS/EWF/main/datasets/UE_Bier.csv',index_col=0)

In [ ]:
# Erstellung einer Pivot Tabelle mit Statistiken
df.loc[:,['einkommen','bildungsgrad']].groupby('bildungsgrad').agg(['count','mean','std','sem'])

In [ ]:
# Abhängig vom Typ der Variablen, müssen diese noch explizit in kategoriale Variablen übersetzt werden! 
df['bildungsgrad_cat'] = df.bildungsgrad.astype('category')
print(df.bildungsgrad.dtype)
print(df.bildungsgrad_cat.dtype)

In [ ]:
# Alternative Lösung, die nur reguläre statsmodels Packages verwendet
# mod = ols('einkommen ~ bildungsgrad_cat', data = df).fit()
# anova_table = anova_lm(mod, typ=2)
# anova_table

In [ ]:
# Durchführung der ANOVA
res = stat()
res.anova_stat(df, res_var='einkommen', anova_model='einkommen ~ bildungsgrad_cat')
res.anova_summary

In [ ]:
# Erstellung von Bins (Gruppierung der Altervariablen)
bins = [0,26,31,36,41,46,51,56,61,66,110]
labels = ['<=25','26-30','31-35','36-40','41-45','46-50','51-55','56-60','61-65', '66+']

df['age Group'] = pd.cut(df['alter'], bins= bins, labels=labels, right=False)
df.head()

In [ ]:
# Erstellung eines einfachen Pivot Tables
contingency = pd.crosstab(df['age Group'], df['männlich'])
print(contingency)

In [ ]:
# Durchführung des Chi-2 Tests
g, p, dof, expctd = scipy.stats.chi2_contingency(contingency)
print('Chi-square = ' + str(g) + '\ndf = ' + str(dof) + '\nSig. = ' + str(p))

## **Übung 7**


In [ ]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np
from statsmodels.formula.api import ols

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/JasperLS/EWF/main/datasets/UE_Bier.csv',index_col=0)
df = df.drop(['Hauptschule','Realschule','Gymnasium','Studium'],axis=1)

In [ ]:
# Erstellung der Korrelationstabelle
df.corr().iloc[:5,:5]

In [ ]:
# Erstellung einer Funktion, die immer den zugehörigen p-Value zurückgibt
def pearsonr_pval(x,y):
  return pearsonr(x,y)[1]

In [ ]:
df.corr(method=pearsonr_pval).iloc[:5,:5].round(3)

In [ ]:
# Dieser Code generiert eine typische Korrelationsmatrix inklusive Sternchen zum Anzeigen des p-Values. Der Code ist etwas komplexer und nicht klausurrelevant
rho = df.corr()
pval = df.corr(method=pearsonr_pval) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
(rho.round(3).astype(str) + p).iloc[:5,:5]

In [ ]:
# Durchführung einer Regression
mod = ols('bierausgaben ~ alter', data = df)
res = mod.fit()
print(res.summary())

In [ ]:
mod = ols('bierausgaben ~ einkommen', data = df)
res = mod.fit()
print(res.summary())

In [ ]:
mod = ols('einkommen ~ bierausgaben', data = df)
res = mod.fit()
print(res.summary())

## **Übung 8**

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
from scipy.stats import pearsonr
from statsmodels.stats.outliers_influence import variance_inflation_factor # Ggf. muss man die Runtime Komplett zurücksetzen, damit dieser Import funktioniert

#### Teil 1 - Mensa

In [ ]:
#Teil 1 
df_mensa = pd.read_csv('https://raw.githubusercontent.com/JasperLS/EWF/main/datasets/UE_Mensa.csv',index_col=0)

In [ ]:
# Ergänzung einer Konstanten
df_mensa['Konstante'] = 1

In [ ]:
df_mensa.sample()

In [ ]:
# Auswahl der un-/abhängigen Variablen
unab_var = ['Konstante','Männlich', 'Essensausgaben_pro_Monat', 'Essensvielfalt', 'Essensqualität', 'Standort1', 'Standort2']
abh_var = 'Zufriedenheit_Mensa'

In [ ]:
# Durchführung der Regression
mod = sm.OLS(df_mensa.loc[:,abh_var], df_mensa.loc[:,unab_var])
res = mod.fit()
print(res.summary())

#### Teil 2 - Bier

In [ ]:
#Teil 2
df = pd.read_csv('https://raw.githubusercontent.com/JasperLS/EWF/main/datasets/UE_Bier.csv',index_col=0)
df['Konstante'] = 1

In [ ]:
# Regression mit einkommen
unab_var_b1 = ['Konstante', 'einkommen', 'alter', 'männlich', 'Realschule',  'Gymnasium', 'Studium']
abh_var_b = 'bierausgaben'
mod = sm.OLS(df.loc[:,abh_var_b], df.loc[:,unab_var_b1])
res = mod.fit()
print(res.summary())

# Regression ohne Einkommen
unab_var_b2 = ['Konstante','alter', 'männlich', 'Realschule',  'Gymnasium', 'Studium']
mod = sm.OLS(df.loc[:,abh_var_b], df.loc[:,unab_var_b2])
res = mod.fit()
print(res.summary())

In [ ]:
df.corr().iloc[:8,:8].round(3)

In [ ]:
def pearsonr_pval(x,y):
  return pearsonr(x,y)[1]

In [ ]:
df.corr(method=pearsonr_pval).iloc[:8,:8].round(3)

In [ ]:
# Dieser Code generiert eine typische Korrelationsmatrix inklusive Sternchen zum Anzeigen des p-Values. Der Code ist etwas komplexer und nicht klausurrelevant
rho = df.corr()
pval = df.corr(method=pearsonr_pval) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
(rho.round(3).astype(str) + p).iloc[:8,:8]

In [ ]:
# Diese Funktion gibt Multikollinearitätswerte aufbereitet zurück
def multicollinearity(X):
    vif = pd.DataFrame()
    vif["VIF_factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["Tolerance"] = 1/vif["VIF_factor"] 
    vif.insert(0,'Variable',X.columns)
    return vif

In [ ]:
# Aufruf der Funktionm, um VIF Werte und Toleranzen zu erhalten
multicollinearity(df[unab_var_b1]).round(3)

In [ ]:
multicollinearity(df[unab_var_b2]).round(3)

# **Übung 9**

In [ ]:
import pandas as pd
import statsmodels.api as sm 

In [ ]:
# Einlesen der Datei 'bier_conjoint_dataset.csv' und hinzufügen einer Spalte 'Konstante' mit 1sen als Zellwerte
df = pd.read_csv('https://raw.githubusercontent.com/JasperLS/EWF/main/datasets/UE_Bier_Conjoint.csv',index_col=0)
df['Konstante'] = 1

In [ ]:
df.head()

In [ ]:
# Festlegen der abhängigen und unabhängigen Variablen
unabh_variablen = ['Konstante','Dunkelbier','Weissbier','NullProzent','Plastikflasche','Glasflasche','FünfzigCent','FünfundsiebzigCent']
abh_variable = 'Preference'

In [ ]:
# Regression ausführen
mod = sm.OLS(df.loc[:,abh_variable], df.loc[:,unabh_variablen])
res = mod.fit()
print(res.summary())